
### 8.10 让属性具有惰性求值的能力

我们想将一个只读的属性定义为property属性方法，只有在访问它时才会参与计算。  
但是，一旦访问了该属性，我们希望把计算的值缓存起来，不要每次访问它时都重新计算。

定义一个惰性属性最有效的方式就是利用描述符类来完成。

In [1]:
class lazyproperty:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            value = self.func(instance)
            setattr(instance, self.func.__name__, value)
            return value
        
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
        
    @lazyproperty
    def area(self):
        print('Computing area')
        return math.pi * self.radius ** 2
    
    @lazyproperty
    def perimeter(self):
        print('Computing perimeter')
        return 2 * math.pi * self.radius
    

In [2]:
c = Circle(4.0)
c.radius

4.0

In [3]:
c.area

Computing area


50.26548245743669

In [4]:
c.area

50.26548245743669

### 8.11 简化数据结构的初始化过程

我们编写了许多类，把他们当作数据结构来用。但是我们厌倦了编写高度重复且样式相同的__init__（）函数。  
通常我们可以将初始化数据结构的步骤归纳到一个单独的__init__（）函数中，并将其定义在一个公共的基类中。

In [1]:
class Structure:
    # Class variable that specifies expected fields
    _fields = []
    def __init__(self, *args, **kwargs):
        if len(args) > len(self._fields):
            raise TypeError('Expected {} arguments'.format(len(self._fields)))
            
        # Set all of the positional arguments
        for name, value in zip(self._fields, args):
            setattr(self, name, value)
            
        # Set the remaining keyword arguments
        for name in self._fields[len(args):]:
            setattr(self, name, kwargs.pop(name))
            
        #  Check for any remaining unknown arguments
        if kwargs:
            raise TypeError('Invalid argument(s): {}'.format(','.join(kwargs)))
            
# Example use
class Stock(Structure):
    _fields = ['name', 'shares', 'price']
    
s1 = Stock('ACME', 50, 91.1)
s2 = Stock('ACME', 50, price=91.1)

###  8.12 定义一个接口或抽象基类

In [2]:
from abc import ABCMeta, abstractmethod

class Istream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass
    
    @abstractmethod
    def write(self, data):
        pass
    

In [3]:
# 抽象基类是给其他类当作基类使用的，这些子类需要实现基类中要求得那些方法。
# 抽象基类也允许其他类向其注册， 然后实现所需得接口。
import io
Istream.register(io.IOBase)

io.IOBase

###  8.13 实现一种数据模型或类型系统

In [ ]:
# Base class. Uses a descriptor to set a value
class Descriptor:
    def __init__(self, name=None, **opts):
        self.name = name
        for key, value in opts.items():
            setattr(self, key, value)
            
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
        
# Descriptor fo enforcing types
class Typed(Descriptor):
    expected_type = type(None)
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError('expected ' + str(self.expected_type))
            super().__set__(instance, value)
            
# Descriptor for enforcing values
class Unsigned(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise TypeError('Expected >= 0')
        super().__set__(instance, value)
        
class MaxSized(Descriptor):
    def __init__(self, name=None, **opts):
        if 'size' not in opts:
            raise TypeError('missing size option')
        super().__init__(name, **opts)
        
    def __set__(self, instance, value):
        if len(value) >= self.size:
            raise ValueError('size must be < ' + str(self.size))
        super().__set__(instance, value)
        
class Integer(Typed):
    expected_type = int
    
class UnsignedInteger(Integer, Unsigned):
    pass

class Float(Typed):
    expected_type = float
    
class UnsignedFloat(Float, Unsigned):
    pass

class String(Typed):
    expected_type = str
    
class SizedString(String, MaxSized):
    pass

class Stock:
    # Specify constraints
    name = SizedString('name', size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        
        

可以运用